In [1]:
from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.utils import utils
import json
import pandas as pd
from tqdm import tqdm
from paths import path_data, path_out_data
import numpy as np
import time

In [2]:
file_test = "round_1_template.csv"
data_test = pd.read_csv("%s/%s"%(path_data,file_test))

data = pd.read_csv("%s/%s"%(path_out_data,"DTC_train.csv"))
InchiKeys = data.standard_inchi_key.unique()


In [3]:
len(InchiKeys)
bad_keys = []

In [4]:
pbar = tqdm(enumerate(InchiKeys,1), total = len(InchiKeys))
descriptor_all = {}
molecule = new_client.molecule

for i, InchiKey in pbar: 
    try:
        SMILE = molecule.get(InchiKey)['molecule_structures']['canonical_smiles']

        chemical = utils.smiles2ctab(SMILE)

        log_p = json.loads(utils.logP(chemical))[0]
        descriptors = json.loads(utils.descriptors(chemical))[0]
        descriptors["log_p"] = log_p

        descriptor_all[InchiKey] =  descriptors
    except:
        bad_keys.append(InchiKey)
    


100%|██████████| 967/967 [1:05:05<00:00,  4.38s/it]


In [7]:
np.save("%s/descriptors/%s"%(path_out_data,"key_descriptor_train.npy"), descriptor_all)

In [5]:
len(descriptor_all)

959

In [6]:
len(bad_keys)

8

In [8]:
# read in test data
file_test = "round_1_template.csv"
data_test = pd.read_csv("%s/%s"%(path_data,file_test))

smiles = set(data_test.Compound_SMILES)
bad_smiles = []

In [9]:
pbar = tqdm(enumerate(smiles,1), total = len(smiles))
descriptor_all = {}
molecule = new_client.molecule

for i, smile in pbar: 
    try:
        

        chemical = utils.smiles2ctab(smile)

        log_p = json.loads(utils.logP(chemical))[0]
        descriptors = json.loads(utils.descriptors(chemical))[0]
        descriptors["log_p"] = log_p

        descriptor_all[smile] =  descriptors
    except:
        bad_smiles.append(smile)


100%|██████████| 70/70 [00:00<00:00, 304.27it/s]


In [10]:
np.save("%s/descriptors/%s"%(path_out_data,"smile_descriptor_test.npy"), descriptor_all)